# 第 2 阶段：RAG 知识库 + 文档问答

本 Notebook 用于学习和调试第 2 阶段的核心功能：
- 文档加载（PDF、Markdown、HTML）
- 文本分块策略
- Embedding 向量化
- 向量存储与索引管理
- 文档检索
- RAG Agent 问答

**参考文档**: `docs/stage_02/`  
**参考脚本**: `scripts/test_rag_query.py`, `scripts/rag_cli.py`

## 1. 环境准备

In [ ]:
# 添加项目根目录到 Python 路径
import sys
from pathlib import Path

backend_dir = Path.cwd()
if backend_dir.name == 'notebooks':
    backend_dir = backend_dir.parent

if str(backend_dir) not in sys.path:
    sys.path.insert(0, str(backend_dir))

print(f"✅ 项目根目录: {backend_dir}")

# 导入必要的模块
from config import settings, get_logger
logger = get_logger(__name__)

## 2. 文档加载

### 2.1 准备测试文档

In [ ]:
# 创建测试文档目录
test_docs_dir = backend_dir / "data" / "documents" / "test"
test_docs_dir.mkdir(parents=True, exist_ok=True)

# 创建一个测试 Markdown 文档
test_md_file = test_docs_dir / "test_document.md"
test_content = """# 机器学习基础

## 什么是机器学习？

机器学习（Machine Learning, ML）是人工智能的一个分支，它使计算机系统能够从数据中学习和改进，而无需明确编程。

## 机器学习的类型

### 1. 监督学习
监督学习使用标记的训练数据来学习输入和输出之间的映射关系。常见算法包括：
- 线性回归
- 逻辑回归
- 决策树
- 随机森林
- 支持向量机（SVM）

### 2. 无监督学习
无监督学习处理未标记的数据，寻找数据中的模式和结构。常见算法包括：
- K-means 聚类
- 层次聚类
- 主成分分析（PCA）
- 自编码器

### 3. 强化学习
强化学习通过与环境交互来学习最优策略，以最大化累积奖励。

## 深度学习

深度学习是机器学习的一个子领域，使用多层神经网络来学习数据的层次化表示。

### 常见的深度学习架构
- 卷积神经网络（CNN）- 用于图像处理
- 循环神经网络（RNN）- 用于序列数据
- 长短期记忆网络（LSTM）- 改进的 RNN
- Transformer - 用于自然语言处理

## 应用场景

机器学习在许多领域都有广泛应用：
- 计算机视觉：图像识别、目标检测
- 自然语言处理：机器翻译、情感分析
- 推荐系统：个性化推荐
- 语音识别：语音助手
- 自动驾驶：环境感知和决策
"""

with open(test_md_file, 'w', encoding='utf-8') as f:
    f.write(test_content)

print(f"✅ 创建测试文档: {test_md_file}")
print(f"   文档大小: {len(test_content)} 字符")

### 2.2 加载文档

In [ ]:
from rag.loaders import load_documents

# 加载文档
documents = load_documents(str(test_docs_dir))

print(f"✅ 加载文档数量: {len(documents)}")
print(f"\n第一个文档预览:")
print(f"  来源: {documents[0].metadata.get('source', 'unknown')}")
print(f"  内容长度: {len(documents[0].page_content)} 字符")
print(f"  内容预览: {documents[0].page_content[:200]}...")

## 3. 文本分块

In [ ]:
from rag.splitters import get_text_splitter

# 创建文本分块器
text_splitter = get_text_splitter(
    chunk_size=500,  # 每块500字符
    chunk_overlap=50  # 重叠50字符
)

# 分块
chunks = text_splitter.split_documents(documents)

print(f"✅ 分块完成")
print(f"   原始文档数: {len(documents)}")
print(f"   分块后数量: {len(chunks)}")
print(f"   块大小设置: {500} 字符")
print(f"   重叠大小: {50} 字符")

# 查看前几个块
print("\n前3个文本块:")
for i, chunk in enumerate(chunks[:3], 1):
    print(f"\n--- 块 {i} ---")
    print(f"长度: {len(chunk.page_content)} 字符")
    print(f"内容: {chunk.page_content[:150]}...")

## 4. Embedding 向量化

In [ ]:
from rag.embeddings import get_embeddings

# 创建 Embedding 模型
embeddings = get_embeddings()

print(f"✅ Embedding 模型创建成功")
print(f"   模型: {settings.embedding_model}")

# 测试 embedding
test_text = "机器学习是人工智能的一个分支"
test_vector = embeddings.embed_query(test_text)

print(f"\n测试文本: {test_text}")
print(f"向量维度: {len(test_vector)}")
print(f"向量前5个值: {test_vector[:5]}")

## 5. 向量存储与索引

### 5.1 创建向量存储

In [ ]:
from rag.vector_stores import create_vector_store

# 创建向量存储
print("🔄 创建向量存储...")
vector_store = create_vector_store(chunks, embeddings)

print(f"✅ 向量存储创建成功")
print(f"   存储的文档块数: {len(chunks)}")

### 5.2 保存索引

In [ ]:
from rag.index_manager import IndexManager

# 创建索引管理器
index_manager = IndexManager()

# 保存索引
index_name = "ml_basics"
index_manager.save_index(vector_store, index_name)

print(f"✅ 索引已保存: {index_name}")
print(f"   索引路径: {index_manager.get_index_path(index_name)}")

### 5.3 加载索引

In [ ]:
# 加载已保存的索引
loaded_vector_store = index_manager.load_index(index_name, embeddings)

print(f"✅ 索引加载成功: {index_name}")

### 5.4 列出所有索引

In [ ]:
# 列出所有可用索引
all_indexes = index_manager.list_indexes()

print(f"📚 可用索引列表 ({len(all_indexes)} 个):")
for idx in all_indexes:
    print(f"  - {idx}")

## 6. 文档检索

### 6.1 相似度搜索

In [ ]:
# 测试相似度搜索
query = "什么是深度学习？"
k = 3  # 返回最相似的3个文档

print(f"🔍 查询: {query}")
print(f"   返回数量: {k}\n")

results = vector_store.similarity_search(query, k=k)

print(f"✅ 找到 {len(results)} 个相关文档:\n")
for i, doc in enumerate(results, 1):
    print(f"--- 文档 {i} ---")
    print(f"内容: {doc.page_content[:200]}...")
    print(f"来源: {doc.metadata.get('source', 'unknown')}")
    print()

### 6.2 带分数的相似度搜索

In [ ]:
# 带相似度分数的搜索
results_with_scores = vector_store.similarity_search_with_score(query, k=k)

print(f"🔍 查询: {query}\n")
print(f"✅ 相似度搜索结果（带分数）:\n")

for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"--- 文档 {i} (相似度: {score:.4f}) ---")
    print(f"内容: {doc.page_content[:150]}...")
    print()

### 6.3 创建检索器

In [ ]:
from rag.retrievers import create_retriever

# 创建检索器
retriever = create_retriever(vector_store, k=4)

print(f"✅ 检索器创建成功")
print(f"   检索数量: 4")

# 使用检索器
retrieved_docs = retriever.invoke(query)

print(f"\n🔍 检索结果: {len(retrieved_docs)} 个文档")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n文档 {i}:")
    print(f"  {doc.page_content[:100]}...")

## 7. RAG Agent 问答

### 7.1 创建 RAG Agent

In [ ]:
from rag.rag_agent import create_rag_agent

# 创建 RAG Agent
rag_agent = create_rag_agent(retriever)

print("✅ RAG Agent 创建成功")

### 7.2 基础问答

In [ ]:
from rag.rag_agent import query_rag_agent

# 测试问题
questions = [
    "什么是机器学习？",
    "监督学习有哪些常见算法？",
    "深度学习有哪些常见架构？",
    "机器学习有哪些应用场景？"
]

for i, question in enumerate(questions, 1):
    print(f"\n{'='*60}")
    print(f"问题 {i}: {question}")
    print(f"{'='*60}\n")
    
    result = query_rag_agent(rag_agent, question)
    
    print(f"回答:\n{result['answer']}")
    print(f"\n参考文档数: {len(result.get('source_documents', []))}")

### 7.3 查看引用来源

In [ ]:
# 详细查看一个问题的回答和来源
question = "什么是深度学习？它有哪些应用？"

print(f"🔍 问题: {question}\n")

result = query_rag_agent(rag_agent, question)

print(f"{'='*60}")
print("回答:")
print(f"{'='*60}")
print(result['answer'])

print(f"\n{'='*60}")
print("引用来源:")
print(f"{'='*60}")

for i, doc in enumerate(result.get('source_documents', []), 1):
    print(f"\n来源 {i}:")
    print(f"  文件: {doc.metadata.get('source', 'unknown')}")
    print(f"  内容: {doc.page_content[:200]}...")

## 8. 完整的索引构建流程

In [ ]:
# 演示完整的索引构建流程
def build_index_from_directory(docs_path: str, index_name: str):
    """从文档目录构建索引的完整流程"""
    
    print(f"\n{'='*60}")
    print(f"构建索引: {index_name}")
    print(f"文档路径: {docs_path}")
    print(f"{'='*60}\n")
    
    # 1. 加载文档
    print("1️⃣  加载文档...")
    documents = load_documents(docs_path)
    print(f"   ✅ 加载 {len(documents)} 个文档\n")
    
    # 2. 文本分块
    print("2️⃣  文本分块...")
    splitter = get_text_splitter(
        chunk_size=settings.chunk_size,
        chunk_overlap=settings.chunk_overlap
    )
    chunks = splitter.split_documents(documents)
    print(f"   ✅ 分块为 {len(chunks)} 个文本块\n")
    
    # 3. 创建 Embedding
    print("3️⃣  创建 Embedding...")
    embeddings = get_embeddings()
    print(f"   ✅ 使用模型: {settings.embedding_model}\n")
    
    # 4. 创建向量存储
    print("4️⃣  创建向量存储...")
    vector_store = create_vector_store(chunks, embeddings)
    print(f"   ✅ 向量存储创建完成\n")
    
    # 5. 保存索引
    print("5️⃣  保存索引...")
    manager = IndexManager()
    manager.save_index(vector_store, index_name)
    print(f"   ✅ 索引已保存\n")
    
    print(f"{'='*60}")
    print(f"✅ 索引构建完成: {index_name}")
    print(f"{'='*60}\n")
    
    return vector_store

# 使用示例
# vector_store = build_index_from_directory(
#     str(test_docs_dir),
#     "my_custom_index"
# )

## 9. 高级功能

### 9.1 自定义分块策略

In [ ]:
# 测试不同的分块参数
chunk_configs = [
    {"chunk_size": 300, "chunk_overlap": 30},
    {"chunk_size": 500, "chunk_overlap": 50},
    {"chunk_size": 1000, "chunk_overlap": 100},
]

print("测试不同的分块策略:\n")

for config in chunk_configs:
    splitter = get_text_splitter(**config)
    chunks = splitter.split_documents(documents)
    
    print(f"块大小: {config['chunk_size']}, 重叠: {config['chunk_overlap']}")
    print(f"  → 生成 {len(chunks)} 个文本块")
    print(f"  → 平均块大小: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f} 字符\n")

### 9.2 多索引管理

In [ ]:
# 演示如何管理多个索引
manager = IndexManager()

# 列出所有索引
indexes = manager.list_indexes()
print(f"当前索引列表 ({len(indexes)} 个):\n")

for idx_name in indexes:
    # 检查索引是否存在
    exists = manager.index_exists(idx_name)
    path = manager.get_index_path(idx_name)
    
    print(f"📚 {idx_name}")
    print(f"   存在: {exists}")
    print(f"   路径: {path}")
    print()

### 9.3 删除索引

In [ ]:
# 删除测试索引（谨慎使用）
# manager.delete_index("ml_basics")
# print("✅ 索引已删除")

print("⚠️  取消注释上面的代码以删除索引")

## 10. 总结

通过本 Notebook，你已经学习了：

✅ 文档加载（支持多种格式）  
✅ 文本分块策略与参数调优  
✅ Embedding 向量化  
✅ 向量存储的创建与管理  
✅ 索引的保存、加载和删除  
✅ 文档检索（相似度搜索）  
✅ RAG Agent 的创建与问答  
✅ 引用来源的追踪  
✅ 多索引管理

**下一步**: 学习第 3 阶段 - LangGraph 工作流（`stage_03_workflow.ipynb`）